In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import pandas as pd
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
from random import randint
import random
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor 
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve, auc
from bayes_opt import BayesianOptimization
import optunity
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_random_more(n):
    mt_x_1 = np.random.rand(n,5)
    mt_x_integar = np.random.choice([18,20,22,23,24,26], size = n,p=[0.1,0.2,0.3,0.1,0.1,0.2]).reshape(-1,1)
    mt_x_b1 = np.random.binomial(1, 0.4,size = n).reshape(-1,1)
    mt_x_tri1 = np.random.binomial(1, 0.5,size = n).reshape(-1,1)
    mt_x_tri2 = np.random.binomial(1, 0.8,size = n).reshape(-1,1)
    for i in range(n):
        if mt_x_tri1[i] + mt_x_tri2[i] > 1:
            mt_x_tri2[i] = 0
    mt_x = np.concatenate((mt_x_1, mt_x_integar,mt_x_b1,mt_x_tri1,mt_x_tri2),axis = 1)
    mt_y = np.zeros(n).reshape(n,1)
    mt_yp = np.zeros(n).reshape(n,1)   
    for i in range(n):
        mt_yp[i]=math.exp(0.6*mt_x[i,0]-8*mt_x[i,1]-4*mt_x[i,2]+0.4*mt_x[i,5]-7*mt_x[i,6]+2*mt_x[i,7]-4*mt_x[i,8])/(1+math.exp(0.6*mt_x[i,0]-8*mt_x[i,1]-4*mt_x[i,2]+0.4*mt_x[i,5]-7*mt_x[i,6]+2*mt_x[i,7]-4*mt_x[i,8]))
        mt_y[i]= np.random.binomial(1, mt_yp[i],size = 1)
#         print(mt_yp[i])
    mt_data = np.concatenate((mt_x, mt_y),axis=1)
    mt_data = pd.DataFrame(mt_data)
    return mt_data

In [3]:
def black_box_function(n_estimators, max_depth):  # min_samples_split, max_features,
    xg_bayes = XGBClassifier(n_estimators=int(n_estimators),
#                                 min_samples_split=int(min_samples_split),
#                                 max_features=min(max_features, 0.999),  # float
                                max_depth=int(max_depth),
                                random_state=2,
                             use_label_encoder=False).fit(X_train, y_train)
    xg_bayes_predict = xg_bayes.predict_proba(X_test)
    res = 1 / log_loss(y_test, xg_bayes_predict)
    return res
pbounds= {'n_estimators': (10, 300),
#          'min_samples_split': (2, 25),
#          'max_features': (0.1, 0.999),
         'max_depth': (5, 30)}

In [4]:
# 未调参
roc_auc_value = 0
accuracy_value = 0
f1_value = 0
recall_value = 0
precision_value = 0
#网格
# wg_r2_value = 0
# wg_mape_value = 0
# wg_rmse_value = 0
# wg_mae_value = 0

# 贝叶斯
bayes_roc_auc_value = 0
bayes_accuracy_value = 0
bayes_f1_value = 0
bayes_recall_value = 0
bayes_precision_value = 0

#粒子群
particle_roc_auc_value = 0
particle_accuracy_value = 0
particle_f1_value = 0
particle_recall_value = 0
particle_precision_value = 0

xg = XGBClassifier(use_label_encoder=False)

for i in range(50):
    # 未调参
    np.random.seed(i)
    random_100 = get_random_more(100)
    X = random_100.iloc[:,:-1]
    y = random_100.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    
    xg.fit(X_train, y_train)
    y_predict = xg.predict(X_test)
    roc_auc_score_ = roc_auc_score(y_test,y_predict)
    accuracy_score_ = accuracy_score(y_test,y_predict)
    f1_score_ = f1_score(y_test, y_predict)
    recall_score_ = recall_score(y_test, y_predict)
    precision_score_ = precision_score(y_test, y_predict)
    
    roc_auc_value += roc_auc_score_
    accuracy_value += accuracy_score_
    f1_value += f1_score_
    recall_value += recall_score_
    precision_value += precision_score_
    
    # 网格
#     parameters_range = dict(n_estimators=[i for i in range(10, 300, 10)],
#                             max_depth=[x for x in range(5,31)])
#     gs = GridSearchCV(xg, parameters_range,cv=5,verbose=2,n_jobs=-1)
#     search = gs.fit(X_train, y_train)
#     xg_opt = search.best_estimator_
#     xg_opt.fit(X_train, y_train)
#     xg_y_predict = xg_opt.predict(X_test)
    
#     wg_r2_score = metrics.r2_score(y_test,xg_y_predict)
#     wg_mape_score = mape(y_test,xg_y_predict)
#     wg_rmse_score = np.sqrt(metrics.mean_squared_error(y_test, xg_y_predict))
#     wg_mae_score = metrics.mean_absolute_error(y_test, xg_y_predict)
#     wg_r2_value += wg_r2_score
#     wg_mape_value += wg_mape_score
#     wg_rmse_value += wg_rmse_score
#     wg_mae_value += wg_mae_score
    
   
    # 贝叶斯
    optimizer = BayesianOptimization(
                f=black_box_function,
                pbounds=pbounds,
                verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
                random_state=1,)
    optimizer.maximize(init_points=5,  #执行随机搜索的步数
                        n_iter=30,)   #执行贝叶斯优化的步数
#     bayes_r2 = optimizer.max["target"]
    bayes_max_depth = optimizer.max["params"]["max_depth"]
    bayes_n_estimators = optimizer.max["params"]["n_estimators"]
    bayes_model = XGBClassifier(max_depth = int(bayes_max_depth), n_estimators = int(bayes_n_estimators),
                                use_label_encoder=False)
    bayes_model.fit(X_train, y_train)
    xg_bayes_predict = bayes_model.predict(X_test)
    
    bayes_roc_auc_score_ = roc_auc_score(y_test,xg_bayes_predict)
    bayes_accuracy_score_ = accuracy_score(y_test,xg_bayes_predict)
    bayes_f1_score_ = f1_score(y_test, xg_bayes_predict)
    bayes_recall_score_ = recall_score(y_test, xg_bayes_predict)
    bayes_precision_score_ = precision_score(y_test, xg_bayes_predict)
    
    bayes_roc_auc_value += bayes_roc_auc_score_
    bayes_accuracy_value += bayes_accuracy_score_
    bayes_f1_value += bayes_f1_score_
    bayes_recall_value += bayes_recall_score_
    bayes_precision_value += bayes_precision_score_
    
      
    # 粒子群
    
    optimal_rbf_pars, info, _ = optunity.maximize(black_box_function,
                                                      num_evals=20,
                                                      solver_name='particle swarm',
                                                      n_estimators=[10,300],
        #                                               min_samples_split=[2,25],
        #                                               max_features = [0.1, 0.999],
                                                      max_depth =  [5, 30]) # default: 'particle swarm'

    #     print(optimal_rbf_pars)

        # print(info)
        # print(_)
    particle_n_estimators = optimal_rbf_pars["n_estimators"]
    particle_max_depth = optimal_rbf_pars["max_depth"]
    particle_model = XGBClassifier(n_estimators = int(particle_n_estimators), max_depth = int(particle_max_depth),
                                   use_label_encoder=False)
    particle_model.fit(X_train,y_train)
    xg_particle_predict = particle_model.predict(X_test)
    
    particle_roc_auc_score_ = roc_auc_score(y_test,xg_particle_predict)
    particle_accuracy_score_ = accuracy_score(y_test,xg_particle_predict)
    particle_f1_score_ = f1_score(y_test, xg_particle_predict)
    particle_recall_score_ = recall_score(y_test, xg_particle_predict)
    particle_precision_score_ = precision_score(y_test, xg_particle_predict)
                                             
                                             
#     particle_r2_score = info.optimum
    # df = optunity.call_log2dataframe(info.call_log)
    # print(df.sort_values('value', ascending=False)[:10])
    particle_roc_auc_value += particle_roc_auc_score_
    particle_accuracy_value += particle_accuracy_score_
    particle_f1_value += particle_f1_score_
    particle_recall_value += particle_recall_score_
    particle_precision_value += particle_precision_score_
    
    
un_roc_auc= roc_auc_value / 50
un_accuracy = accuracy_value / 50
un_f1 = f1_value / 50
un_recall = recall_value / 50
un_precision = precision_value / 50

# wg_r2= wg_r2_value / 50
# wg_mape = wg_mape_value / 50
# wg_rmse = wg_rmse_value / 50
# wg_mae = wg_mae_value / 50

bayes_roc_auc= bayes_roc_auc_value / 50
bayes_accuracy = bayes_accuracy_value / 50
bayes_f1 = bayes_f1_value / 50
bayes_recall = bayes_recall_value / 50
bayes_precision = bayes_precision_value / 50

particle_roc_auc= particle_roc_auc_value / 50
particle_accuracy = particle_accuracy_value / 50
particle_f1 = particle_f1_value / 50
particle_recall = particle_recall_value / 50
particle_precision = particle_precision_value / 50


[00:13:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
[00:13:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  1        |  2.412    |  15.43    |  218.9    |
[00:13:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

[00:13:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  22       |  2.336    |  5.149    |  62.59    |
[00:13:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  23       |  2.46     |  30.0     |  162.9    |
[00:13:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like

[00:13:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:13:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:13:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:13:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

KeyboardInterrupt: 

In [ ]:
random_100 = pd.DataFrame([[un_roc_auc, un_accuracy, un_f1, un_recall,un_precision],
             [bayes_roc_auc,bayes_accuracy,bayes_f1,bayes_recall,bayes_precision],
             [particle_roc_auc,particle_accuracy,particle_f1,particle_recall,particle_precision],
             [0,0,0,0,0]],
             columns=["roc_auc","accuracy","f1","recall","precision"],index = ["未调参","贝叶斯优化","粒子群","遗传算法"])

In [ ]:
# random_100